# Installing the NAG library and running this notebook

This notebook depends on the NAG library for Python to run. Please read the instructions in the [Readme.md](https://github.com/numericalalgorithmsgroup/NAGPythonExamples/blob/master/local_optimization/Readme.md#install) file to download, install and obtain a licence for the library.

Instruction on how to run the notebook can be found [here](https://github.com/numericalalgorithmsgroup/NAGPythonExamples/blob/master/local_optimization/Readme.md#jupyter).

# Edit and solve different variants of a production-planning problem

## Correct Rendering of this notebook

This notebook makes use of the `latex_envs` Jupyter extension for equations and references.  If the LaTeX is not rendering properly in your local installation of Jupyter , it may be because you have not installed this extension.  Details at https://jupyter-contrib-nbextensions.readthedocs.io/en/latest/nbextensions/latex_envs/README.html

The notebook is also not rendered well by GitHub so if you are reading it from there, you may prefer the [pdf version instead](./static/portfolio_optimization_qcqp.pdf).

## Problem description
We consider a situation where a factory can manufacture two different chemicals $A_1$ and $A_2$. The goal for the factory is to determine the quantities $x_1$ and $x_2$ of each chemical to maximize profit under the following circumstances:
- a unit of $A_1$ weighs 40kg and a unit of $A_2$ weighs 80kg;
- the total daily production cannot exceed 16000kg to match the transport capabilities;
- the factory generates \\$2 profit for each unit of $A_1$ and \\$4.5 profit for each unit of $A_2$;
- both products need to use the same machine as part of their respective processes; a unit of A1 requires 1.2 minutes of machine time while a unit of $A_2$ requires 3 minutes; the machine can only function for 1500 minutes daily;
- a unit of $A_1$ uses 6 square metres of packing material while a unit of $A_2$ uses 10 square metres; 6000 square metres of packing materials are available each day;
- production of $A_2$ is limited to 100 units per day.

Note that since the chemicals are considered fluid, the quantities $x_1$ and $x_2$ are not limited to integer values.\\

The problem can be formulated as a linear program:
\vspace{0.1cn}
\begin{equation*}
\begin{array}{lll}
\underset{x\in\Re^n}{\mbox{maximize}} & 2x_1+4.5x_2&\\[0.6ex]
\mbox{subject to} & 1.2x_1 + 3x_2 \leq 1500, &\text{ (machine time constraint)} \\[0.6ex]
     & 6x_1+10x_2 \leq 6000, & \text{ (packaging material constraint)} \\[0.6ex]
     & 40x_1+80x_2 \leq 16000, & \text{ (transport constraint)} \\[0.6ex]
     & 0 \leq x_1, & \text{ (capacity constraint)} \\[0.6ex]
     & 0 \leq x_1 \leq 100 & \text{ (capacity constraint)} \\[0.6ex]
\end{array}
\end{equation*}

In [15]:
# Define and solve the linear program with the NAG optimization modelling suite

from naginterfaces.library import opt
from naginterfaces.base import utils

infbnd = 1.0e20
    
# Initialize the Optimization model handle with the number of variables
handle = opt.handle_init(2)

# Define a linear objective function
opt.handle_set_linobj(handle, cvec=[2.0, 4.5])

# Box constraints
opt.handle_set_simplebounds(
    handle,
    bl=[0.0, 0.0],
    bu=[infbnd, 100])

# Set the linear constraints
opt.handle_set_linconstr(
    handle,
    bl=[-infbnd, -infbnd, -infbnd],
    bu=[1500.0, 6000.0, 16000.0],
    irowb=[1, 1, 2, 2, 3, 3],
    icolb=[1, 2, 1, 2, 1, 2],
    b=[1.2, 3.0, 6.0, 10.0, 40.0, 80.0]
)

# Set some alogorithmic options
for option in [
        'Print Options = NO',
        'Print Level = 1',
        'Task = Max',
        'Print Solution = X',
]:
    opt.handle_opt_set(handle, option)

# Use an explicit I/O manager for abbreviated iteration output:
iom = utils.FileObjManager(locus_in_output=False)

# Solve the problem
res = opt.handle_solve_lp_ipm(handle, io_manager=iom)

 E04MT, Interior point method for LP problems
 Status: converged, an optimal solution found
 Final primal objective value  8.500000E+02
 Final dual objective value    8.500000E+02

 Primal variables:
   idx   Lower bound       Value       Upper bound
     1   0.00000E+00    2.00000E+02         inf
     2   0.00000E+00    1.00000E+02    1.00000E+02


The optimal repartition is to produce 200 units of $A_1$ and 100 units of $A_2$ for a total profit of 850\\$.

### Plant expansion: a new chemical $A_3$ can be produced 

The following data is available for $A_3$:
- a unit of $A_3$ takes 5 minutes on the common machine;
- a unit of $A_3$ takes 12 square metres of packaging material;
- a unit of $A_3$ weighs 120kg;
- a unit of $A_3$ generates \\$7 profit;
- production of $A_3$ is limited to 50 units per day.

The problem becomes:
\vspace{0.1cn}
\begin{equation*}
\begin{array}{lll}
\underset{x\in\Re^n}{\mbox{maximize}} & 2x_1+4.5x_2+7x_3&\\[0.6ex]
\mbox{subject to} & 1.2x_1 + 3x_2 + 5x_3 \leq 1500, &\text{ (machine time constraint)} \\[0.6ex]
     & 6x_1+10x_2+12x_3 \leq 6000, & \text{ (packaging material constraint)} \\[0.6ex]
     & 40x_1+80x_2+120x_3 \leq 16000, & \text{ (transport constraint)} \\[0.6ex]
     & 0 \leq x_1, & \text{ (capacity constraint)} \\[0.6ex]
     & 0 \leq x_1 \leq 100 & \text{ (capacity constraint)} \\[0.6ex]
     & 0 \leq x_3 \leq 50 & \text{ (capacity constraint)} \\[0.6ex]
\end{array}
\end{equation*}

In [16]:
# Edit the problem to account for the new plant capacity
# add a variable
opt.handle_add_vars(handle, nadd=1)

# Box Constraint on the new variable
opt.handle_set_bound(handle, comp='X', idx=3, bli=0.0, bui=50.0)

# Add the linear objective component
opt.handle_set_linobj_coeff(handle, idxci=3, ci=7.0)

# Add linear constraints coefficients
opt.handle_set_linconstr_coeff(handle, idlc=1, icolbj=3, bij=5.0)
opt.handle_set_linconstr_coeff(handle, idlc=2, icolbj=3, bij=12.0)
opt.handle_set_linconstr_coeff(handle, idlc=3, icolbj=3, bij=120.0)

# Solve the problem again
res = opt.handle_solve_lp_ipm(handle, io_manager=iom)

 E04MT, Interior point method for LP problems
 Status: converged, an optimal solution found
 Final primal objective value  9.000000E+02
 Final dual objective value    9.000000E+02

 Primal variables:
   idx   Lower bound       Value       Upper bound
     1   0.00000E+00    5.00000E+01         inf
     2   0.00000E+00    1.00000E+02    1.00000E+02
     3   0.00000E+00    5.00000E+01    5.00000E+01


The new optimum is to produce 50 units of $A_1$, 100 units of $A_2$ and 50 units of $A_3$ for a total profit of 900\\$.

### New regulation: additional constraint

At a later date, regulation changes require that products $A_2$ and $A_3$ follow a rigorous quality assurance test before being sent to market. Now the factory is only able to process a total of 100 units per day which amounts to adding the following constraint to our linear program:
\begin{equation*}
\begin{array}{ll}
     x_2+x_3 \leq 100 & \text{ (regulation constraint)} \\[0.6ex]
\end{array}
\end{equation*}

In [17]:
# Add a linear constraint
opt.handle_set_linconstr(
    handle,
    bl=[-infbnd],
    bu=[100.0],
    irowb=[1, 1],
    icolb=[2, 3],
    b=[1.0, 1.0]
)

# Solve the problem again
res = opt.handle_solve_lp_ipm(handle, io_manager=iom)

 E04MT, Interior point method for LP problems
 Status: converged, an optimal solution found
 Final primal objective value  8.750000E+02
 Final dual objective value    8.750000E+02

 Primal variables:
   idx   Lower bound       Value       Upper bound
     1   0.00000E+00    1.50000E+02         inf
     2   0.00000E+00    5.00000E+01    1.00000E+02
     3   0.00000E+00    5.00000E+01    5.00000E+01


With the new regulation, maximum profit of 875\\$ can be achived by producing 150 units of $A_1$, 50 units of $A_2$ and 50 units of $A_3$.

In [18]:
opt.handle_free(handle)